# MODELING SUDOKU AS AN INTEGER LINEAR PROGRAMMING PROBLEM

Importing the relevant libraries:

In [1]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

In [2]:
from mip import *
import numpy as np
from sudoku.utils import N, STEP, objective_grid

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


Sample grid for testing:

In [3]:
grid_example = np.array([[0, 0, 5, 3, 0, 0, 0, 0, 0],
                         [8, 0, 0, 0, 0, 0, 0, 2, 0],
                         [0, 7, 0, 0, 1, 0, 5, 0, 0],
                         [4, 0, 0, 0, 0, 5, 3, 0, 0],
                         [0, 1, 0, 0, 7, 0, 0, 0, 6],
                         [0, 0, 3, 2, 0, 0, 0, 8, 0],
                         [0, 6, 0, 5, 0, 0, 0, 0, 9],
                         [0, 0, 4, 0, 0, 0, 0, 3, 0],
                         [0, 0, 0, 0, 0, 9, 7, 0, 0]], dtype='int8')

Preprocess the grid to become the model input:

In [4]:
def preprocess_grid(grid):
    return [(ij//N, ij%N, k) for ij, k in enumerate(grid.flatten())]

In [5]:
preprocess_grid(grid_example)

[(0, 0, 0),
 (0, 1, 0),
 (0, 2, 5),
 (0, 3, 3),
 (0, 4, 0),
 (0, 5, 0),
 (0, 6, 0),
 (0, 7, 0),
 (0, 8, 0),
 (1, 0, 8),
 (1, 1, 0),
 (1, 2, 0),
 (1, 3, 0),
 (1, 4, 0),
 (1, 5, 0),
 (1, 6, 0),
 (1, 7, 2),
 (1, 8, 0),
 (2, 0, 0),
 (2, 1, 7),
 (2, 2, 0),
 (2, 3, 0),
 (2, 4, 1),
 (2, 5, 0),
 (2, 6, 5),
 (2, 7, 0),
 (2, 8, 0),
 (3, 0, 4),
 (3, 1, 0),
 (3, 2, 0),
 (3, 3, 0),
 (3, 4, 0),
 (3, 5, 5),
 (3, 6, 3),
 (3, 7, 0),
 (3, 8, 0),
 (4, 0, 0),
 (4, 1, 1),
 (4, 2, 0),
 (4, 3, 0),
 (4, 4, 7),
 (4, 5, 0),
 (4, 6, 0),
 (4, 7, 0),
 (4, 8, 6),
 (5, 0, 0),
 (5, 1, 0),
 (5, 2, 3),
 (5, 3, 2),
 (5, 4, 0),
 (5, 5, 0),
 (5, 6, 0),
 (5, 7, 8),
 (5, 8, 0),
 (6, 0, 0),
 (6, 1, 6),
 (6, 2, 0),
 (6, 3, 5),
 (6, 4, 0),
 (6, 5, 0),
 (6, 6, 0),
 (6, 7, 0),
 (6, 8, 9),
 (7, 0, 0),
 (7, 1, 0),
 (7, 2, 4),
 (7, 3, 0),
 (7, 4, 0),
 (7, 5, 0),
 (7, 6, 0),
 (7, 7, 3),
 (7, 8, 0),
 (8, 0, 0),
 (8, 1, 0),
 (8, 2, 0),
 (8, 3, 0),
 (8, 4, 0),
 (8, 5, 9),
 (8, 6, 7),
 (8, 7, 0),
 (8, 8, 0)]

Modeling the problem:
- Binary variables identified by (i, j, k);
- (i, j, k) represents whether position i (row) and j (column) is filled with value k;
- Objective function is omitted as it is irrelevant to the problem, since sudoku has a unique solution.

In [6]:
def create_model(C):  # set C represents the clues given by the game
    
    model = Model(sense=MINIMIZE, solver_name=CBC)

    x = [[[model.add_var(var_type=BINARY, name="x" + str(i) + str(j) + str(k))
           for k in range(1, N+1)]  # range of names between {1,..., 9} != range of positions between {0,...,8}
          for j in range(N)]
         for i in range(N)]
    
    # Game clues must be strictly followed
    for i, j, k in C:
        if k:  # 0 represents that it is not filled
            model += x[i][j][k-1] == 1, "CLUE_x" + str(i) + str(j) + str(k)
    
    # All cells must be filled in necessarily once
    for i in range(N):
        for j in range(N):
            model += xsum(x[i][j][k] for k in range(N)) == 1, "FILL_x" + str(i) + str(j)
    
    # AllDiff constraints on lines
    for i in range(N):
        for k in range(N):
            model += xsum(x[i][j][k] for j in range(N)) == 1, "ALLDIFF_line" + str(i) + str(k)
    
    # AllDiff constraints on columns
    for j in range(N):
        for k in range(N):
            model += xsum(x[i][j][k] for i in range(N)) == 1, "ALLDIFF_column" + str(j) + str(k)
    
    # AllDiff constraints on 3x3 squares
    for m, n in zip([yy//STEP*STEP for yy in range(N)], [xx for xx in range(0, N, STEP)]*STEP):
        for k in range(N):
            model += xsum(x[i][j][k] for i in range(m, m+STEP)
                          for j in range(n, n+STEP)) == 1, "ALLDIFF_square" + str(m//STEP + n//STEP) + str(k)

    return model

In [7]:
def solver_IP(grid):
    goal_grid = grid.copy()  # does not change the grid passed as a parameter
    
    model = create_model(preprocess_grid(goal_grid))
    model.optimize()

    for v in model.vars:
        if v.x:  # was assigned with 1
            i, j, k = map(int, v.name[1:])  # ignore the character 'x'

            if not goal_grid[i, j]:
                goal_grid[i, j] = k

    return goal_grid

Testing the solver:

In [8]:
goal_grid = solver_IP(grid_example)

goal_grid

Welcome to the CBC MILP Solver 
Version: devel 
Build Date: Nov 15 2020 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 189 (-158) rows, 185 (-544) columns and 760 (-2179) elements
Clp1000I sum of infeasibilities 3.43475e-06 - average 1.81733e-08, 7 fixed columns
Coin0506I Presolve 173 (-16) rows, 168 (-17) columns and 709 (-51) elements
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.012, Presolve 0.00, Idiot 0.01

Starting MIP optimization
Cgl0003I 6 fixed, 0 tightened bounds, 0 strengthened rows, 15 substitutions
Cgl0004I processed model has 173 rows, 170 columns (170 integer (1

array([[1, 4, 5, 3, 2, 7, 6, 9, 8],
       [8, 3, 9, 6, 5, 4, 1, 2, 7],
       [6, 7, 2, 9, 1, 8, 5, 4, 3],
       [4, 9, 6, 1, 8, 5, 3, 7, 2],
       [2, 1, 8, 4, 7, 3, 9, 5, 6],
       [7, 5, 3, 2, 9, 6, 4, 8, 1],
       [3, 6, 7, 5, 4, 2, 8, 1, 9],
       [9, 8, 4, 7, 6, 1, 2, 3, 5],
       [5, 2, 1, 8, 3, 9, 7, 6, 4]], dtype=int8)

In [9]:
# Checking if the objective grid was reached
# Using objective_grid function from sudoku package

objective_grid(goal_grid)

True